In [1]:
import pandas as pd
import numpy as np
from requests import get
from bs4 import BeautifulSoup as bs

import src.cleaning as cleaning

In [2]:
trucks = pd.read_csv('data/truck_transfers.csv')
april = pd.read_csv('data/april_2014.csv')

In [12]:
# stations = cleaning.stations_coordinates (april)
# stations capacity
# trucks = cleaning.coordinate_columns (trucks)
# trucks = cleaning.truck_trips_coordinates (stations, trucks)
# trucks = cleaning.time_difference (trucks)

In [21]:
first = april.bike_id.unique().tolist()[0]
others = april.bike_id.unique().tolist()[1:10]

ALL_TRIPS = cleaning.bike_human_truck_trips (april, first)

In [23]:
for bike_id in others:
    cleaning.bike_human_truck_trips(april, bike_id).head(1)